<a href="https://www.kaggle.com/code/kelixirr/natural-language-processing-with-disaster-tweets?scriptVersionId=144807492" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.target.value_counts()

In [ ]:
len(train_df) + len(test_df)

In [ ]:
# shuffle the training data 
train_data = train_df.sample(frac =1, random_state = 42)
train_data.head()

In [ ]:
# visulize the data
import random
random_index = random.randint(0, len(train_df) - 5)

for row in train_data[["text", "target"]][random_index:random_index + 5].itertuples():
    _,text, target = row
    print(f"Target: {target}", "(real disater)" if target > 0 else "(not real disater)")
    print(f"Text: \n {text} \n")
    print("...\n")
    

In [ ]:
# split the data inot training and validation set
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data["text"].to_numpy(),
                                                                           train_data["target"].to_numpy(),
                                                                           test_size =0.1,
                                                                           random_state = 42)

In [ ]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

In [ ]:
train_sentences[:10]

In [ ]:
train_labels[:10]

## Text Vectorization

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [ ]:
# average number of words or tokens in training data
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

In [ ]:
text_vectorizer = TextVectorization(max_tokens = 10000,
                                    output_mode = "int",
                                    output_sequence_length = 15)

In [ ]:
# fit the text vectorizer to data
text_vectorizer.adapt(train_sentences)

In [ ]:
# checking for the text vectorizer 
random_sentence = random.choice(train_sentences)
print(f"original {random_sentence}")
print(f"Vectorized: {text_vectorizer([random_sentence])}")

In [ ]:
# unique words in our dictionary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5 = words_in_vocab[:5]
bottom_5 = words_in_vocab[-5:]

print("Vocab", len(words_in_vocab))
print("Top 5 words:", top_5)
print("Bottom 5 words:", bottom_5)

## Creating Embeddings

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = 10000,
                            output_dim = 128,
                            embeddings_initializer = "uniform",
                            input_length = 15,
                            name = "embedding_1")
embedding

In [ ]:
# example from sample data

random_sentence = random.choice(train_sentences)
print(f"original {random_sentence}")
print(f"Vectorized: {text_vectorizer([random_sentence])}")
print(f"Vectorized: {embedding(text_vectorizer([random_sentence]))}")

## Creating Our Models

### 0. Naive Bayes - Baseline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenizartiona and modelling the pipeline
model_0 = Pipeline([
    
    ("tfdf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)
baseline_score

In [ ]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

#### Creating Evaluation functions for our future use

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    
    # accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    
    # model_precision,f1_score, recall using weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    
    model_results = {
        
        "accurracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1
    }
    
    return model_results

In [ ]:
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

### Model 1 - Simple Dense

In [ ]:
# importing helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
# create directory to save log 
SAVE_DIR = "model_logs"

In [ ]:
# building the model 
inputs = layers.Input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name = "model_1_choice")

In [ ]:
# compile the model
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_1.summary()

In [ ]:
# fit the model 
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

In [ ]:
model_1.evaluate(val_sentences, val_labels)

In [ ]:
embedding.weights

In [ ]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

In [ ]:
# make predictions
model_1_preds = tf.squeeze(tf.round(model_1.predict(val_sentences)))
model_1_preds[:10]

In [ ]:
# model_1 metrics 
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)

model_1_results

In [ ]:
# comparing results of baseline with new models 

def compare_with_baseline(baseline_results, new_model_results):
    for key, value in baseline_results.items():
        print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key] - value:.2f}")


In [ ]:
compare_with_baseline(baseline_results=baseline_results, 
                                new_model_results=model_1_results)

## Model 2 - LSTM 

In [ ]:
tf.random.set_seed(42)

model_2_embedding = layers.Embedding(input_dim = 10000,
                                     output_dim = 128,
                                     embeddings_initializer = "uniform",
                                     input_length = 15,
                                     name = "embedding_2")

# LSTM Model
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs , name = "model_2_LSTM")

In [ ]:
# compile the model 

model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

model_2.summary()

In [ ]:
# fit the model 
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]


In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

In [ ]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

In [ ]:
compare_with_baseline(baseline_results, model_2_results)

### Model 3 -- GRU

In [ ]:
model_3_embedding = layers.Embedding(input_dim = 10000,
                                     output_dim = 128,
                                     embeddings_initializer = "uniform",
                                     input_length = 15,
                                     name = "embedding_3")

# model 
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name = "model_3_GRU")

In [ ]:
model_3.compile( loss = "binary_crossentropy",
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics = ["accuracy"])

In [ ]:
model_3.summary()

In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels, 
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

In [ ]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

In [ ]:
compare_with_baseline(baseline_results, model_3_results)

### Bidirection RNN 

In [ ]:
model_4_embedding = layers.Embedding(input_dim = 10000,
                                     output_dim = 128,
                                     embeddings_initializer="uniform",
                                     input_length = 15,
                                     name = "embedding_4")

# model

inputs = layers.Input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name = "model_4_Bidrectional")
model_4.summary()

In [ ]:
model_4.compile( loss = "binary_crossentropy",
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics = ["accuracy"])

In [ ]:
model_4_history = model_4.fit(train_sentences,
                              train_labels, 
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

In [ ]:
compare_with_baseline(baseline_results, model_4_results)

### CNN1D

In [ ]:
model_5_embedding = layers.Embedding(input_dim = 10000,
                                     output_dim = 128,
                                     embeddings_initializer = "uniform",
                                     input_length = 15,
                                     name = "embedding_5")


# model 
inputs = layers.Input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters = 32, kernel_size = 5, activation = "relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")
model_5.summary()


In [ ]:
model_5.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, 
                                                                     "Conv1D")])

In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

In [ ]:
model_5_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_5_preds)
model_5_results

In [ ]:
compare_with_baseline(baseline_results, model_5_results)

### Model 6: TensorFlow Hub Pretrained Sentence Encoder

In [ ]:
import tensorflow_hub as hub

In [ ]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [ ]:
model_6 = tf.keras.Sequential([
    
    sentence_encoder_layer,
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = "relu"),
], name = 'model_6')

model_6.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

model_6.summary()

In [ ]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, 
                                                                     "tf_hub_sentence_encoder")])

In [ ]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

In [ ]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

In [ ]:
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

In [ ]:
compare_with_baseline(baseline_results, model_6_results)

### Model 7: TensorFlow Hub Pretrained Sentence Encoder 10% of the training data

In [ ]:
# split our data into 10% training
import numpy as np
train_90, train_10, train_labels_90, train_labels_10 = train_test_split(np.array(train_sentences),
                                                                        train_labels,
                                                                        test_size=0.1,
                                                                        random_state=42)
   
                                           

In [ ]:
len(train_sentences), len(train_10)

In [ ]:
pd.Series(train_labels_10).value_counts()

In [ ]:
# model 7 
model_7 = tf.keras.models.clone_model(model_6)

model_7.compile(loss = "binary_crossentropy",
                optimizer = "Adam",
                metrics = ["accuracy"])

model_7.summary()

In [ ]:
model_7_history = model_7.fit(x=train_10,
                              y=train_labels_10,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "10_percent_tf_hub_sentence_encoder")])

In [ ]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

In [ ]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

In [ ]:
model_7_results = calculate_results(val_labels, model_7_preds)
model_7_results

In [ ]:
compare_with_baseline(baseline_results, model_7_results)

## Comparing Results Of All Models

In [ ]:
all_model_results = pd.DataFrame({
    "Baseline": baseline_results,
    'Simple Dense': model_1_results,
    "LSTM": model_2_results,
    "GRU": model_3_results,
    "Bidirectional": model_4_results,
    "Conv1D": model_5_results,
    "TF Sentence Encoder": model_6_results,
    "TF 10 Percent Data": model_7_results,
})

all_model_results = all_model_results.transpose()
all_model_results

In [ ]:
all_model_results["accurracy"] = all_model_results["accurracy"]/100

In [ ]:
all_model_results.plot(kind="bar", figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0));

In [ ]:
all_model_results.sort_values("f1", ascending=False)["f1"].plot(kind="bar", figsize=(10, 7));

## Model Ensembling 

In [ ]:
baseline_pred_probs = np.max(model_0.predict_proba(val_sentences), axis = 1)

#combining prediction probabilities
combined_pred_probs = baseline_pred_probs + tf.squeeze(model_2_pred_probs, axis=1) + tf.squeeze(model_6_pred_probs)

#averaging the prediction
combined_preds = tf.round(combined_pred_probs/3)
combined_preds[:20]                       

In [ ]:
ensemble_results = calculate_results(val_labels, combined_preds)
ensemble_results

In [ ]:
ensemble_results["accurracy"] = ensemble_results["accurracy"] / 100
ensemble_results["accurracy"] 

In [ ]:
all_model_results.loc["Ensemble Results"] = ensemble_results

In [ ]:
all_model_results

In [ ]:
val_df = pd.DataFrame({"text": val_sentences,
                       "target": val_labels,
                       "pred": model_7_preds,
                       "pred_prob": tf.squeeze(model_7_pred_probs)})
val_df.head()

In [ ]:
# most wronged predictions
most_wrong = val_df[val_df["target"] != val_df["pred"]].sort_values("pred_prob", ascending=False)
most_wrong[:10]

In [ ]:
# False positives Most Wrong
for row in most_wrong[:10].itertuples():
    _, text, target, pred, prob = row
    print(f"Target: {target}, Pred: {int(pred)}, Prob: {prob}")
    print(f"Text:\n{text}\n")
    print("----\n")

In [ ]:
# false negatives
for row in most_wrong[-10:].itertuples():
    _, text, target, pred, prob = row
    print(f"Target: {target}, Pred: {int(pred)}, Prob: {prob}")
    print(f"Text:\n{text}\n")
    print("----\n")

## Making Prediction On Test Data

In [ ]:
test_df.head()

In [ ]:
# making prediction on test data
test_sentences = test_df["text"].to_list()
test_samples = random.sample(test_sentences, 10)

for test_sample in test_samples:
    pred_prob = tf.squeeze(model_6.predict([test_sample]))
    pred = tf.round(pred_prob)
    print(f"Pred: {int(pred)}, Prob: {pred_prob}")
    print(f"Text:\n{test_sample}\n")
    print("----\n")
  

In [ ]:
test_df.head()

In [ ]:
pred_prob = tf.squeeze(model_6.predict(test_df["text"].to_numpy()))
pred = tf.round(pred_prob)
pred_prob[:10]

In [ ]:
pred[:10]

In [ ]:
test_data = test_df[:]

In [ ]:
test_data.head()

In [ ]:
prediction = pd.DataFrame({
    
    "id": test_data["id"],
    "target": pred
})

In [ ]:
prediction.to_csv("/kaggle/working/prediction.csv", index = False)